In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import statsmodels.formula.api as sm

In [ ]:
data = pd.read_csv('stata_data.csv')
data.head()

In [ ]:
data.describe()

In [ ]:
onemresult = sm.ols(formula="one_m_diff ~ NYSE + CI", data=data).fit()
print(onemresult.summary())

In [ ]:
threemresult = sm.ols(formula="three_m_diff ~ NYSE + CI", data=data).fit()
print(threemresult.summary())

In [ ]:
sixmresult = sm.ols(formula="six_m_diff ~ NYSE + CI", data=data).fit()
print(sixmresult.summary())

In [ ]:
oneyresult = sm.ols(formula="one_y_diff ~ NYSE + CI", data=data).fit()
print(oneyresult.summary())